In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import drive

LOAD DATASET

In [4]:
# Muat Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Muat dataset
data = []
labels = []
DIRECTORY = "/content/drive/MyDrive/Images/Dataset"
CATEGORIES = ["Mask", "NoMask"]
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)

    	data.append(image)
    	labels.append(category)

TRAIN DATASET

In [ ]:
# Encode dan ubah format label dan data
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
# Membagi data untuk train dan test
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [ ]:
# Augmentasi data
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# Arsitektur model
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# Bentuk output model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# Model utama
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
# Nilai Learning Rate, Epochs, dan Batch Size
LR = 1e-4
EPOCHS = 20
BS = 32

In [ ]:
# Initial layer
for layer in baseModel.layers:
	layer.trainable = False

# Optimisasi Adam
opt = Adam(lr=LR, decay=LR / EPOCHS)

# Konfigurasi model untuk dilatih
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Latih model sebanyak jumlah epochs/iterasi
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

Epoch 1/20
2/2 [==============================] - 4s 534ms/step - loss: 0.8915 - accuracy: 0.5319
Epoch 2/20
2/2 [==============================] - 1s 200ms/step - loss: 0.7711 - accuracy: 0.5957
Epoch 3/20
2/2 [==============================] - 1s 450ms/step - loss: 0.6930 - accuracy: 0.6094
Epoch 4/20
2/2 [==============================] - 1s 206ms/step - loss: 0.7517 - accuracy: 0.6170
Epoch 5/20
2/2 [==============================] - 1s 187ms/step - loss: 0.7051 - accuracy: 0.5745
Epoch 6/20
2/2 [==============================] - 1s 197ms/step - loss: 0.6365 - accuracy: 0.6596
Epoch 7/20
2/2 [==============================] - 1s 201ms/step - loss: 0.5306 - accuracy: 0.7660
Epoch 8/20
2/2 [==============================] - 1s 211ms/step - loss: 0.5802 - accuracy: 0.7447
Epoch 9/20
2/2 [==============================] - 1s 483ms/step - loss: 0.5075 - accuracy: 0.8723
Epoch 10/20
2/2 [==============================] - 1s 191ms/step - loss: 0.5138 - accuracy: 0.7447
Epoch 11/20
2/2 [==

CREATE MODEL/CLASSIFIER

In [ ]:
# Simpan model
model.save("mask_detector.model", save_format="h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
